#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
#import scipy as sp
#import matplotlib as mpl
#import matplotlib.pyplot as plt
#from pathlib import Path
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [2]:
#Read the entire Benthiemer sandstone
global image_1, label_1
#change the path to the correct directory before running the code
# please note: the dimensions of the image is 800 x 800 x 800
image_1 = sitk.ReadImage('D:\\Ankita PhD\\pyradiomics\\data\\bentheimer_nrrd_8bit_800cube.nrrd')

#### Extraction Settings

In [3]:
settings = {}
settings['label'] = 255
settings['binWidth'] = 1

#### Log messages

In [12]:
log_file = 'D:\\Ankita PhD\\pyradiomics\\notebooks\\log_bentheimer_8bit_attempt12.txt'
handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
handler.setFormatter(formatter)
radiomics.logger.addHandler(handler)

# Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
# Verbositiy level, the logger level will also determine the amount of information printed to the output
radiomics.logger.setLevel(logging.DEBUG)

In [13]:
%%time
# the size of the mask should be the same as that of the image
mask_x = 800
mask_y = 800
mask_z = 800

v1 = []
v2 = []
v3 = []
v4 = []
v5 = []
v6 = []
v7 = []
v8 = []
v9 = []
v10 = []
v11 = []
v12 = []
v13 = []
v14 = []
v15 = []
v16 = []

# here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
# sizes increase with every iteration
u_window_size = [10]#, 20, 30, 40, 50, 60, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700]
global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
for window_size in u_window_size:  
    
    i_loc = int((mask_x - window_size)/2)
    j_loc = int((mask_y - window_size)/2)
    k_loc = int((mask_z - window_size)/2)
    
    # Generate the mask
    full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
    full_mask[i_loc:i_loc+window_size,j_loc:j_loc+window_size,k_loc:k_loc+window_size] = 255
    #print("Window Location", str(i_loc), str(i_loc+window_size), str(j_loc), str(j_loc+window_size), str(k_loc), str(k_loc+window_size))
    label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
    label_1.CopyInformation(image_1)
    del full_mask

    bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
    if correctedMask is not None:
        label_1 = correctedMask
    croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
    del correctedMask, label_1

    #calculate secondorder stats
    glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
    glszmFeatures.enableAllFeatures()
    print('Enabled GLSZM features...',)
    result = glszmFeatures.execute()

    print('Calculated GLSZM features: ')
    for (key, val) in six.iteritems(result):
        if key =='GrayLevelNonUniformity':
            v1.append(val)
        if key =='GrayLevelNonUniformityNormalized':
            v2.append(val)
        if key =='GrayLevelVariance':
            v3.append(val)
        if key =='HighGrayLevelZoneEmphasis':
            v4.append(val)
        if key == 'LargeAreaEmphasis':
            v5.append(val)
        if key == 'LargeAreaHighGrayLevelEmphasis':
            v6.append(val)
        if key == 'LargeAreaLowGrayLevelEmphasis':
            v7.append(val)
        if key == 'LowGrayLevelZoneEmphasis':
            v8.append(val) 
        if key == 'SizeZoneNonUniformity':
            v9.append(val)
        if key == 'SizeZoneNonUniformityNormalized':
            v10.append(val)
        if key == 'SmallAreaEmphasis':
            v11.append(val)
        if key == 'SmallAreaHighGrayLevelEmphasis':
            v12.append(val)
        if key == 'SmallAreaLowGrayLevelEmphasis':
            v13.append(val)
        if key == 'ZoneEntropy':
            v14.append(val)
        if key == 'ZonePercentage':
            v15.append(val)
        if key == 'ZoneVariance':
            v16.append(val)

print("Completed Domain Calculations for Window ", str(window_size), ". Stored all the values too.")

Enabled GLSZM features...
Calculated GLSZM features: 
Completed Domain Calculations for Window  10 . Stored all the values too.
Wall time: 5.24 s


In [14]:
my_df = pd.DataFrame(list(zip(u_window_size, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16)))
my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance"})

,Window Sizes,GrayLevelNonUniformity,GrayLevelNonUniformityNormalized,GrayLevelVariance,HighGrayLevelZoneEmphasis,LargeAreaEmphasis,LargeAreaHighGrayLevelEmphasis,LargeAreaLowGrayLevelEmphasis,LowGrayLevelZoneEmphasis,SizeZoneNonUniformity,SizeZoneNonUniformityNormalized,SmallAreaEmphasis,SmallAreaHighGrayLevelEmphasis,SmallAreaLowGrayLevelEmphasis,ZoneEntropy,ZonePercentage,ZoneVariance
0,10,5.48,0.2192,121.86240000000001,271.68,19456.24,892640.28,569.3150838003149,0.3637935038246308,3.96,0.1584,0.31584257973016955,92.71704170381952,0.07850399721257906,4.213660689688179,0.025,17856.24


In [ ]:
name_csv = 'glszm_bentheimer_8bit_dc_attempt11.csv'
my_df.to_csv(name_csv, index=False, header=True)